In [1]:
import sys
sys.path.append('../_amigocloud')

In [4]:
import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

In [5]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [20]:
id = 12

In [24]:
# consulta pata extraer los datos de inspeccion por id concreto
query = {'query': 'select ca.canhero, ca.maquina, ca.modelo, ca.codigo_maquina, ca.frente, insp.* from dataset_341171 insp\
    inner join dataset_341170 ca on ca.amigo_id=insp.maquina_ref_preven\
    where insp.id={id_insp}'.format(id_insp=id)}
project = 'https://app.amigocloud.com/api/v1/projects/34080/sql'
select = amigocloud.get(project, query)
data = select['data'][0]

In [25]:
# modificacion de la fecha a formato dd/mm/aaaa
data['fecha_registro'] = datetime.strptime(data['fecha_registro'], "%Y-%m-%d %H:%M:%S%z").strftime("%d/%m/%Y")

In [27]:
# modificacion del nombre de las claves, con una k_ por delante, por que los campos booblenos (1_1, 2_2, etc) no se pueden trabajar bien
data = {'k_'+key : value for (key, value) in data.items()}

In [31]:
insp = collections.namedtuple("insp", data.keys())(*data.values())

In [32]:
insp

insp(k_canhero='515 / AGUILERA OLGA RIVERO VDA. DE', k_maquina='JOHN DEERE', k_modelo='3510', k_codigo_maquina=4, k_frente=4, k_fecha_registro='02/04/2024', k_amigo_id='65fd9c3e440640559764089c507d9aa8', k_id=12, k_usuario='Juan Pablo Jaldin Duran', k_obs='Falta reparar el extractor primario que en unos 15 días estará listo, caja de corte de base está en revisión , falta colocar despuntador ñ, punteras del divisor de línea, el pateador y elevador que ya están revisados solo es colocar a su posición ', k_ubicacion='0101000020E6100000E427D53E1D9F4FC0E7C589AF763A31C0', k_1_1=False, k_1_2=False, k_1_3=False, k_1_4=False, k_1_5=False, k_2_1=False, k_2_2=False, k_2_3=False, k_2_4=False, k_3_1=False, k_3_2=True, k_3_3=True, k_3_4=True, k_3_5=True, k_3_6=True, k_3_7=True, k_3_8=True, k_4_1=True, k_4_2=True, k_4_3=True, k_4_4=True, k_4_5=True, k_5_1=True, k_5_2=True, k_5_3=True, k_5_4=True, k_5_5=True, k_5_6=True, k_6_1=True, k_6_2=True, k_6_3=True, k_6_4=True, k_6_5=True, k_7_1=False, k_7_2=Fa

In [33]:
query_fotos = {'query': 'select * from gallery_57490 where source_amigo_id=\'{id}\''.format(id=insp.k_amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/34080/sql'
fotos = amigocloud.get(project, query_fotos)['data']

In [34]:
fotos

[{'amigo_id': '486687f8d5014e7ba718039f9bae87a9',
  'source_amigo_id': '65fd9c3e440640559764089c507d9aa8',
  'filename': '57490_20240402090618493.jpg',
  's3_filename': '57490_20240402090618493.jpg',
  'file_size': 4749671,
  'mime_type': 'image/jpeg',
  'datetime': '2024-04-02 13:06:23+00:00',
  'amigocloud_url': 'app.amigocloud.com/api/v1/related_tables/57490/files/65fd9c3e440640559764089c507d9aa8/486687f8d5014e7ba718039f9bae87a9/57490_20240402090618493.jpg',
  'location': '0101000020E610000099426B6E1A9F4FC082F865D58C3A31C0'},
 {'amigo_id': 'b9d1a8b1ddcc4939ab85f508ad7eab10',
  'source_amigo_id': '65fd9c3e440640559764089c507d9aa8',
  'filename': '57490_20240402090558709.jpg',
  's3_filename': '57490_20240402090558709.jpg',
  'file_size': 5103099,
  'mime_type': 'image/jpeg',
  'datetime': '2024-04-02 13:06:06+00:00',
  'amigocloud_url': 'app.amigocloud.com/api/v1/related_tables/57490/files/65fd9c3e440640559764089c507d9aa8/b9d1a8b1ddcc4939ab85f508ad7eab10/57490_20240402090558709.jpg',

In [133]:
doc = DocxTemplate(ruta + '/templates/tpl_insp_cosechadoras-preventivo_2.docx')

In [141]:
lista_fotos_inline = []
for foto in fotos:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append(docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['s3_filename'], width=Mm(50)))

In [135]:
len(lista_fotos_inline)

6

In [ ]:
# Datos que quieres que contenga el QR
texto = '''
--UTEA--
--PROGRAMA CAÑA LIMPIA--

Id de Evaluacion: {nro_reg}
Fecha de Evaluacion: {fecha}
Nombre de Canero: {canero}
Codigo Canero: {cod_ca}
Maquina Cosechadora: {maquina}
Codigo de Maquina: {cod_ma}
Calificacion de Evaluacion: {nota} %'''\
.format(
    nro_reg=data_rep['nro_reg'], 
    fecha=data_rep['fecha'], 
    canero=data_rep['nom_ca'],
    cod_ca=data_rep['cod_ca'],
    prop=data_rep['nom_prop'],
    area=data_rep['total'])
    # Generar el código QR
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4)
qr.add_data(texto)
qr.make(fit=True)

img = qr.make_image(fill_color="black", back_color="white")
filename_qr = 'qr_' + str(data_rep['cod_ca']) + '_' + str(data_rep['cod_prop']) + '_' + data_rep['fecha']
ruta_qr = ruta + '/planos/' + filename_qr + '.png'
# Guardar el código QR en un archivo
img.save(ruta_qr)

In [136]:
context = {'insp':insp, 'fotos':lista_fotos_inline}

In [137]:
doc.render(context)

In [138]:
cod_nom = insp.k_canhero.split(' / ')
file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)

In [139]:
file_name

'515_ECP_02-04-2024_AGUILERA OLGA RIVERO VDA. DE_12'

In [140]:
doc.save(ruta + r'/informes/_' + file_name + '.docx')